In [1]:
import arcpy
from arcpy import env
from arcpy.sa import *

In [2]:
arcpy.env.workspace = r"E:\stu1\Projects\POI2023\POI2023.gdb"
arcpy.env.overwriteOutput = True

In [3]:
poi_layer = r"E:\stu1\Projects\POI2023\POI2023.gdb\restaurant23"

In [4]:
# 添加属性字段
arcpy.AddField_management(poi_layer, "GridID", "LONG")
arcpy.AddField_management(poi_layer, "Weight", "FLOAT")

<Result 'E:\\stu1\\Projects\\POI2023\\POI2023.gdb\\restaurant23'>

### 数据分布评估

In [5]:
# 执行最近邻指数分析
nni_result = arcpy.AverageNearestNeighbor_stats(poi_layer, "EUCLIDEAN_DISTANCE", "NO_REPORT", None)

In [6]:
# 输出NNI结果
print(f"Nearest Neighbor Ratio: {nni_result[0]}")
print(f"z-score: {nni_result[1]}")
print(f"p-value: {nni_result[2]}")

Nearest Neighbor Ratio: .123968
z-score: -529.579655
p-value: 0


In [13]:
# KDE分析的输出路径
output_kde_raster = r"E:\stu1\Projects\POI2023\POI2023.gdb\kde_output"

In [9]:
# 执行核密度估计
kde_result = arcpy.sa.KernelDensity(poi_layer, None)

In [14]:
# 保存输出结果
kde_result.save(output_kde_raster)

print(f"Kernel Density Estimation completed. Output saved to {output_kde_raster}")

Kernel Density Estimation completed. Output saved to E:\stu1\Projects\POI2023\POI2023.gdb\kde_output


### 四叉树自适应划分格网

In [21]:
# 四叉树节点类
class QuadTreeNode:
    node_count = 0  # 用于生成唯一的格网ID
    
    def __init__(self, extent, max_points=1000, max_depth=5, depth=0):
        self.extent = extent  # 节点的空间范围
        self.children = []  # 子节点列表
        self.max_points = max_points  # 每个节点能包含的最大点数
        self.max_depth = max_depth  # 树的最大深度
        self.depth = depth  # 当前节点的深度
        self.points = []  # 此节点包含的POI点列表
        self.grid_id = QuadTreeNode.node_count  # 唯一的格网ID
        QuadTreeNode.node_count += 1
        
    # 分割当前节点为四个子节点
    def split(self):
        xmin, ymin, xmax, ymax = self.extent
        xmid, ymid = (xmin + xmax) / 2, (ymin + ymax) / 2
        
        # 定义四个子区域的空间范围
        extents = [
            (xmin, ymin, xmid, ymid),  # 西南
            (xmid, ymin, xmax, ymid),  # 东南
            (xmin, ymid, xmid, ymax),  # 西北
            (xmid, ymid, xmax, ymax)   # 东北
        ]
    
        # 为每个子区域创建一个新的四叉树节点
        for extent in extents:
            self.children.append(QuadTreeNode(extent, max_points=self.max_points, max_depth=self.max_depth, depth=self.depth + 1))
            
    # 将一个点插入四叉树中，并存储格网ID或节点引用
    def insert(self, point):
        if not self._contains(point):
            return False  # 点不在当前节点的范围内

        if len(self.points) < self.max_points or self.depth == self.max_depth:
            self.points.append(point)  # 如果节点未满，或达到最大深度，直接添加点
            return True

        if not self.children:
            self.split()  # 如果子节点不存在，先分割当前节点

        return any(child.insert(point) for child in self.children)
    
    # 检查点是否在当前节点的范围内
    def _contains(self, point):
        x, y = point
        xmin, ymin, xmax, ymax = self.extent
        return xmin <= x < xmax and ymin <= y < ymax

In [16]:
# 构建四叉树并为每个POI点分配格网ID
def build_quadtree_and_assign_grid_ids(poi_layer, root):
    with arcpy.da.UpdateCursor(poi_layer, ["SHAPE@XY", "GridID"]) as cursor:
        for row in cursor:
            point = row[0]
            if root.insert(point):
                # 如果点被插入到四叉树，更新其格网ID
                row[1] = find_grid_id(root, point)
                cursor.updateRow(row)

In [17]:
# 递归查找包含给定点的节点的格网ID
def find_grid_id(node, point):
    if node._contains(point):
        if not node.children or node.depth == node.max_depth:
            return node.grid_id
        for child in node.children:
            grid_id = find_grid_id(child, point)
            if grid_id is not None:
                return grid_id
    return None

In [18]:
# 根据行政边界矢量数据生成四叉树覆盖的空间范围
boundary_layer = r"E:\stu1\Projects\POI2023\POI2023.gdb\wuhan2023"
boundary_extent = arcpy.Describe(boundary_layer).extent
tree_extent = (boundary_extent.XMin, boundary_extent.YMin, boundary_extent.XMax, boundary_extent.YMax)

需要设置的参数和变量：
* poi_layer
* max_points_per_node
* max_depth
* tree_extent

In [22]:
# 创建四叉树的根节点
max_points_per_node = 100
max_depth = 7
root = QuadTreeNode(tree_extent, max_points=max_points_per_node, max_depth=max_depth)

In [23]:
# 为POI图层中的每个点构建四叉树并分配格网ID
build_quadtree_and_assign_grid_ids(poi_layer, root)

### 去丛聚调整权重

In [24]:
# 计算每个格网内POI的数量
def calculate_grid_poi_counts(poi_layer):
    grid_poi_counts = {}
    with arcpy.da.SearchCursor(poi_layer, ["GridID"]) as cursor:
        for row in cursor:
            grid_id = row[0]
            grid_poi_counts[grid_id] = grid_poi_counts.get(grid_id, 0) + 1
    return grid_poi_counts

In [25]:
# 为每个POI点分配权重
def assign_weights_to_pois(poi_layer, grid_poi_counts):
    # 计算有POI的格网总数
    total_occupied_grids = len(grid_poi_counts)
    # 计算每个格网的权重
    grid_weight = 1.0 / total_occupied_grids if total_occupied_grids else 0
    
    with arcpy.da.UpdateCursor(poi_layer, ["GridID", "Weight"]) as cursor:
        for row in cursor:
            grid_id = row[0]
            poi_count = grid_poi_counts.get(grid_id, 0)
            weight = grid_weight / poi_count if poi_count > 0 else 0
            row[1] = weight
            cursor.updateRow(row)

In [26]:
# 计算每个格网内POI的数量
grid_poi_counts = calculate_grid_poi_counts(poi_layer)

In [27]:
# 为每个POI点分配权重
assign_weights_to_pois(poi_layer, grid_poi_counts)